$$
P(\text{score}|\text{dist}=x) = 1 - \frac{x}{q}, \ q > 50$$

- during each practice run, shoot from distances $x =\overline{1 \dots 50}$ 
- precisely $2 \%$ chance to score a total of exactly $20$ points
- **find $q$, rounded to $10$ decimal places**